<a href="https://colab.research.google.com/github/OksanaTymchenko/BudgetCheck/blob/master/BudgetCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q docx2python

In [0]:
import numpy as np
import pandas as pd
import re
from docx2python import docx2python

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
budget_doc = docx2python('/content/drive/My Drive/Colab Notebooks/budget.docx')
budget = list(filter(lambda x: len(x) > 1, budget_doc.body))
budget = budget[1:9]
# print(doc_result.body[1])
# print(result[1])
# print(result[2][5])
# budget_df = pd.DataFrame(budget[1][1:]).applymap(lambda s: ' '.join(s))
# budget_df.columns = [val[0] for val in budget[1][0]]
# budget_df.head()
# print(df["Редакція, запропонована на перше читання"][1])

In [0]:
def clean_parts(raw_budget):
    temp = pd.DataFrame(raw_budget[1:])
    temp = temp.drop(temp.columns[:-2], axis=1).dropna()
    temp = temp[temp.iloc[:,0].map(lambda d: len(d[0]) > 0)].applymap(lambda s: ' '.join(s))
    temp.columns = ['Правки', 'Висновки']
    return temp


In [0]:
budget_df = list(map(clean_parts, budget))
budget_df = pd.concat(budget_df, axis=0)


In [0]:
def get_edit_number(edit):
    try:
        return re.search('\-(\d+)\-', edit).group()
    except AttributeError:
        return None

In [271]:
def get_deputy(edit):
  pattern = re.compile('(н\.д\.)([а-я А-Я\.]+)(\(Реєстр\.картка №\d+\))', re.IGNORECASE)
  # deputy_list = pattern.search(edit)
  deputy_list = pattern.findall(edit)
  print(deputy_list)
  # print( deputy_list.group(1), deputy_list.group(2), deputy_list.group(3),)

get_deputy(budget_df.loc[1]['Правки'])

[('Н.д.', 'Бойко Ю.А. ', '(Реєстр.картка №350)'), ('Н.д.', 'Медведчук В.В. ', '(Реєстр.картка №352)'), ('Н.д.', 'Славицька А.К. ', '(Реєстр.картка №386)'), ('Н.д.', 'Колтунович О.С. ', '(Реєстр.картка №371)'), ('Н.д.', 'Борт В.П. ', '(Реєстр.картка №377)'), ('Н.д.', 'Павленко Ю.О. ', '(Реєстр.картка №374)'), ('Н.д.', 'Плачкова Т.М. ', '(Реєстр.картка №380)'), ('Н.д.', 'Скорик М.Л. ', '(Реєстр.картка №365)'), ('Н.д.', 'Кальцев В.Ф. ', '(Реєстр.картка №373)'), ('Н.д.', 'Гнатенко В.С. ', '(Реєстр.картка №49)'), ('Н.д.', 'Пономарьов О.С. ', '(Реєстр.картка №78)'), ('Н.д.', 'Мороз В.В. ', '(Реєстр.картка №59)'), ('Н.д.', 'Качний О.С. ', '(Реєстр.картка №385)'), ('Н.д.', 'Королевська Н.Ю. ', '(Реєстр.картка №353)'), ('Н.д.', 'Лукашев О.А. ', '(Реєстр.картка №113)'), ('Н.д.', 'Солод Ю.В. ', '(Реєстр.картка №47)'), ('Н.д.', 'Мамка Г.М. ', '(Реєстр.картка №372)'), ('Н.д.', 'Пузанов О.Г. ', '(Реєстр.картка №381)')]


In [248]:
'''
видатки у розмірі 143.784,2 тис. грн, збільшивши видатки загального фонду на 50 млн грн. 
видатки у розмірі 2.980.852,4 тис. грн, збільшивши відповідно на 1.000.000 тис. грн за рахунок зменшення видатків на Апарат ВРУ...
Передбачити нову бюджетну програму... видатки у розмірі 500.000 тис. грн за рахунок зменшення видатків на Апарат ВРУ...
Передбачити нову бюджетну програму...з обсягом 2.583.900 тис. грн.
Передбачити нову бюджетну програму... в обсязі 107.963,8 тис.грн. Збільшення видатків пропонується здійснити за рахунок збільшення на відповідну суму неподаткових надходжень за кодом... 
за загальним фондом у сумі 250 млн грн та спеціальним фондом - 250 млн грн
Передбачити нову бюджетну програму... з обсягом видатків 3.561.140,1 тис грн, зменшивши видатки ... на 1.172.259,3 тис. грн. та додатково виділивши на нову бюджетну програму видатки в обсязі 2.388.880,8 тис. грн
збільшити видатки
передбачити видатки
передбачити нові бюджетні програми
включити (нову бюджетну програму|до переліку|до Державної програми|до бюджету|в розподіл|у розрахунки
запровадити
виділити
нову бюджетну програму - 77
виділити - 4
'''

keywords = ['збільши(ти|вши)[а-я А-Я]*видатки', 'передбачити(?!.*зменшивши)', 
            'нову бюджетну програму', 'встановити .*збільшивши', 'включити до', 
            'збільшити фінансування', 'виділити', 'включити (в|у) роз', 'встановити .*тобто збільшити']
reg_lst = []
for keyword in keywords:
    reg_lst.append(re.compile(keyword, re.IGNORECASE))

for edit in budget_df["Правки"]:
  for compiled_reg in reg_lst:
    m = compiled_reg.search(edit)
    if m:
      print(get_edit_number(edit), m.group())
      break
 
     

-38- передбачити
-64- збільшити видатки
-65- передбачити
-66- передбачити
-83- передбачити
-93- передбачити
-99- передбачити
-100- передбачити
-112- передбачити
-128- передбачити
-137- збільшити видатки
-153- Передбачити
-157- Передбачити
-180- передбачити
-196- передбачити
-203- передбачити
-211- передбачити
-212- передбачити
-215- передбачити
-266- Передбачити
-268- Передбачити
-271- передбачити
-272- передбачити
-284- передбачити
-285- передбачити
-292- передбачити
-293- передбачити
-295- передбачити
-296- передбачити
-319- Передбачити
-324- Передбачити
-373- встановити рентну плату за користування надрами (код 13030000) у сумі 67.874.943,7 тис. грн (тобто збільшити
-422- передбачити
-424- встановити податок на додану вартість із ввезених на митну територію України товарів (код 140700000) у сумі 362.500.000 тис. грн (тобто збільшити
-426- передбачити
-436- передбачити
-461- передбачити
-467- передбачити
-469- передбачити
-471- нову бюджетну програму
-472- нову бюджетну програму
-473